# Git/GitHub + `bedtools` +TSCC + YOU!

To bring it all together, you will combine everything you've learned so far about UNIX, bash shell, TSCC, git, and github to collaboratively write a submitter script to TSCC.

1. Identify your randomly assigned partner that was emailed out.
2. Pick one person to be reponsible for exercises 1-3 from `6_tf_binding_promoters.ipynb` (person1) and the the other to be responsible for exercises 4-6 (person2)
3. Have person1 create a GitHub repo called `biom262-hw1` and write a short `README.md` file
4. Have person2 add a LICENSE file
5. Collaboratively work on a submitter script called `tf_binding.sh` to TSCC which has all the `##PBS` flags shown in the TSCC Quick Start Guide below, and contribute the lines of code that you are responsible for. Use `git blame tf_binding.sh` to make sure person1 is to "blame" for exercises 1-3 and person2 is to "blame" for exercises 4-5. The "blame" of the remaining file doesn't matter.
6. Your final repository should have three files:

```
LICENSE
README.txt
tf_binding.sh
```

Resources for TSCC:
* [TSCC Quick Start guide](http://www.sdsc.edu/support/user_guides/tscc-quick-start.html)
* [TSCC User Guide](http://www.sdsc.edu/support/user_guides/tscc.html)